# FPUS23 — YOLO Optimized (Balanced) for n/s/m

Trains YOLO11 nano/small/medium on the balanced train set (val/test original).
- Optimized = Balanced train only (YOLO11 is anchor‑free; anchors ignored)
- Pipeline: balance → COCO→YOLO balanced → train n/s/m → save to Drive


In [ ]:
# Save to Drive
from google.colab import drive; drive.mount('/content/drive', force_remount=True)
DRIVE_DIR = '/content/drive/MyDrive/FPUS23_runs'
import pathlib as p; p.Path(DRIVE_DIR).mkdir(parents=True, exist_ok=True)
print('Saving to:', DRIVE_DIR)


In [ ]:
# Recommended batch
import torch, os
gpu = torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'
print('GPU:', gpu)
if 'T4' in gpu: os.environ['FPUS23_BATCH']='16'
elif 'A100' in gpu: os.environ['FPUS23_BATCH']='24'
else: os.environ['FPUS23_BATCH']='12'
print('Recommended batch:', os.environ['FPUS23_BATCH'])


## Build Balanced YOLO train (once per session)

In [ ]:
# Balance COCO then convert to YOLO balanced
!python '/content/fpus23/scripts/balance_fpus23_dataset.py' --coco-root /content/fpus23_project/dataset/fpus23_coco
!python '/content/fpus23/scripts/tools/coco_to_yolo.py' \
  --coco-json /content/fpus23_project/dataset/fpus23_coco/annotations/train_balanced.json \
  --images-dir /content/fpus23_project/dataset/fpus23_coco/images_balanced/train \
  --out-yolo-root /content/fpus23_project/dataset/fpus23_yolo_balanced \
  --orig-data-yaml /content/fpus23_project/dataset/fpus23_yolo/data.yaml
# Quick check
!sed -n '1,40p' /content/fpus23_project/dataset/fpus23_yolo_balanced/data.yaml


## Train — yolo11n (nano) — balanced


In [ ]:
!python '/content/fpus23/scripts/train_yolo_fpus23_phase1.py' \
  --data /content/fpus23_project/dataset/fpus23_yolo_balanced/data.yaml \
  --model yolo11n.pt \
  --epochs 100 \
  --batch ${FPUS23_BATCH:-16} \
  --imgsz 768 \
  --project $DRIVE_DIR \
  --name fpus23_optimized_n


## Train — yolo11s (small) — balanced


In [ ]:
!python '/content/fpus23/scripts/train_yolo_fpus23_phase1.py' \
  --data /content/fpus23_project/dataset/fpus23_yolo_balanced/data.yaml \
  --model yolo11s.pt \
  --epochs 100 \
  --batch ${FPUS23_BATCH:-16} \
  --imgsz 768 \
  --project $DRIVE_DIR \
  --name fpus23_optimized_s


## Train — yolo11m (medium) — balanced (T4 batch 8)


In [ ]:
!python '/content/fpus23/scripts/train_yolo_fpus23_phase1.py' \
  --data /content/fpus23_project/dataset/fpus23_yolo_balanced/data.yaml \
  --model yolo11m.pt \
  --epochs 100 \
  --batch 8 \
  --imgsz 768 \
  --project $DRIVE_DIR \
  --name fpus23_optimized_m
